In [ ]:
#WORKING
#Convert Piutang details ke Aging per 30 hari to >120 hari

import pandas as pd
import numpy as np

#sumber data dari unpaid invoice 
data = pd.read_excel(r'ardetail_20200824.xlsx') 

#data yang diperlukan 
df = pd.DataFrame(data, columns = ['Code','Group','Partner Name','Partner Outlet Def', 'sales', 'AR Person','Tanggal', 'iyear','Total'])

#memberikan nilai 0 ke data kosong. Biar bisa dibaca
df = df.fillna(0)

df.info()

In [ ]:
#Tambah kolom baru "Today" yang otomatis diisi dengan tanggal hari ini
df ['Today']=pd.Timestamp("today")
#format date
df ['Today']=df['Today'].map(lambda ts:ts.strftime("%Y-%m-%d"))
#ganti format tanggal biar bisa dihitung selisihnya
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df['Today'] = pd.to_datetime(df['Today'])

#formula hitung tanggal
df ['Beda'] = df['Today']-df['Tanggal']
#Convert attributes Kolom menjadi Integer
df['Beda'] = df['Beda'].dt.days.astype('int16')
#kondisi untuk Aging per 30 days

#Aging per year and 30 days
conditions = [
    (df['Beda'] < 31),
    (df['Beda'] <= 60) & (df['Beda'] >= 31),
    (df['Beda'] <= 90) & (df['Beda'] >= 61),
    (df['Beda'] <= 120) & (df['Beda'] >= 91),
    (df['Beda'] > 120) & (df['iyear']== 2020),
    (df['Beda'] > 120) & (df['iyear']== 2019),
    (df['Beda'] > 120) & (df['iyear']== 2018),
    (df['Beda'] > 120) & (df['iyear']== 2017),
    (df['Beda'] > 120) & (df['iyear']== 2016),
    ]
choices = ['0-30','31-60','61-90','91-120','>120 2020','2019','2018','2017','2016',]

df['Aging'] = np.select(conditions, choices, default='null')
 
df.info()

df

#Pivot table dari dataframe dibuat per aging 30 hari dan per tahun
pivot1 = df.pivot_table(index=['Group','Code','Partner Name','sales','AR Person'], columns= ['Aging'], values=['Total'], 
                      aggfunc='sum', margins=True)

#Setting kolom biar sesuai urutan yg kita mau
pivot1 = pivot1 [[('Total','0-30'),('Total','31-60'), ('Total','61-90'), ('Total','91-120'),('Total','>120 2020'),('Total','2019'), ('Total','2018'), ('Total','2017'),('Total','2016'),('Total','All')]]

#menghitung total aging selama tahun 2020, dari aging 30 hari hingga diatas 120 hari
pivot1['','2020']=pivot1.iloc[:,0:5].sum(axis=1)

#Setting kolom sesuai urutan, setelah hitungan total aging 2020
pivot = pivot1 [[('Total','0-30'),('Total','31-60'), ('Total','61-90'), ('Total','91-120'),('Total','>120 2020'),('','2020'),('Total','2019'), ('Total','2018'), ('Total','2017'),('Total','2016'),('Total','All')]]

pivot

#Summary data untuk Grup saja 
Grup = df.pivot_table(index=['Group'], columns= ['Aging'], values=['Total'], 
                      aggfunc='sum', margins=True)

Grup = Grup [[('Total','0-30'),('Total','31-60'), ('Total','61-90'), ('Total','91-120'),('Total','>120 2020'),('Total','2019'), ('Total','2018'), ('Total','2017'),('Total','2016'),('Total','All')]]

Grup['','2020']=Grup.iloc[:,0:5].sum(axis=1)

Grup1 = Grup [[('Total','0-30'),('Total','31-60'), ('Total','61-90'), ('Total','91-120'),('Total','>120 2020'),('','2020'),('Total','2019'), ('Total','2018'), ('Total','2017'),('Total','2016'),('Total','All')]]

Grup


#Menghitung subtotal tiap grup 
total = pd.concat([
    d.append(d.sum().rename(('Total','','','',k)))
    for k, d in pivot.groupby('Group')
]).append(pivot.sum().rename(('','Grand','','', 'Total')))

total

#Data khusus grup tertentu
#Grup = total.groupby("Group")

#df1 = Grup.get_group('Cabang LK')

#G1 = pd.concat([
   # df1.append(df1.sum().rename(('Total','','','','','')))
   # ])

#G1 = G1 [[('Total','0-30'),('Total','31-60'), ('Total','61-90'), ('Total','91-120'),('Total','>120 2020'),('','2020'),('Total','2019'), ('Total','2018'), ('Total','2017'),('Total','2016'),('Total','2015'),('Total','All')]]

#G1 = G1.sort_index(ascending=False)

#G1

#total.to_excel (r'Aging ALL 17082020.xlsx', index=True, header=True)

In [ ]:
total.info()

In [ ]:
total.to_excel (r'Aging ALL 24082020.xlsx', index=True, header=True)